# MATH 521 - Numerical Analysis of Differential Equations

Christoph Ortner, 03/2024

## Assignment 4 : Implementation Practice

**Name:**
Jincong Li

**Student ID:**
60539939

### Notes 

This assignment is concerned with implementation and empirical convergence studies. There will be a little bit of theory but it purely formal - no proofs are required. I will not give too detailed instructions or have model codes ready to be completed. But you are of course allowed (in fact encouraged) to use as much of the codes from class as you wish. In fact the assignment requires again relatively minimal adaptation of my model codes.

You may add as many solutions cells as needed to implement your numerical methods and tests. To help with Q2 I advise that you implement it in a structured way based on function calls rather than a single script, so you can then re-use those functions for the convergence study. For the study of singularities, you should be able to import and use the codes from class directly, you should not need to adapt them at all.

In principle you may of course write your own code from scratch using a different (but similarly low-level) finite element package. Unless you already have experience with such a package I don't recommend this since it would be a fairly time-consuming undertaking. If you plan to do this please check with me so I can be sure the code you are using is suitable. 

## Q1: Inhomogeneous Dirichlet Problem [15]

We consider the boundary value problem 
$$\begin{aligned}
    - {\rm div} A(x) \nabla u &= f, \quad \Omega \\ 
          u &= u_D(x), \quad \partial\Omega,
\end{aligned}$$

where $\Omega = (0, 1)^2$, $f(x) = x y$, $u_D(x) = \sin(\pi(x_1+x_2))$, and 
$$
    A(x) = \begin{pmatrix} 
        1 & \frac12 \sin(\pi x_1) \\ 
        \frac12 \sin(\pi x_1) & 1 
    \end{pmatrix}
$$

Adapt the code from class to implement a $P_k$-Finite Element method. Visualize the solution. 

In [6]:
using Pkg; Pkg.activate(".")
using LinearAlgebra, Ferrite, LaTeXStrings, FerriteViz, WGLMakie, Makie
Makie.inline!(true);

  Activating project at `E:\UBC\MEng\Term2\MATH 521\HW4\A4`


In [14]:
import Pkg; Pkg.add("plotly")#master
import plotly.io as pio#master
pio.renderers.default = true #"notebook"

LoadError: The following package names could not be resolved:
 * plotly (not found in project, manifest or registry)
[36m   Suggestions:[39m [0m[1mP[22m[0m[1ml[22m[0m[1mo[22m[0m[1mt[22m[0m[1ml[22m[0m[1my[22m [0m[1mP[22m[0m[1ml[22m[0m[1mo[22m[0m[1mt[22m[0m[1ml[22m[0m[1my[22mJS [0m[1mP[22m[0m[1ml[22m[0m[1mo[22m[0m[1mt[22m[0m[1ml[22m[0m[1my[22mSave [0m[1mP[22m[0m[1ml[22m[0m[1mo[22m[0m[1mt[22m[0m[1ml[22m[0m[1my[22mBase [0m[1mP[22m[0m[1ml[22m[0m[1mo[22m[0m[1mt[22m[0m[1ml[22m[0m[1my[22mLight [0m[1mP[22m[0m[1ml[22m[0m[1mo[22m[0m[1mt[22m[0m[1ml[22m[0m[1my[22mKaleido

In [9]:
grid = generate_grid(Triangle, (10, 10))
FerriteViz.wireframe(grid, markersize = 10, strokewidth = 2,figure = (resolution=(500,500),))

## Q2: Convergence Study [15]

Implement a numerical convergence study of you method from Q1. Using the method of manufactured solutions, demonstrate that - provided your solution is sufficiently smooth - your method achieves the optimal rate of convergence in the **energy norm**, 
$$
  \| u - u_h \|_a := \bigg( \int_\Omega \nabla (u-u_h)^T A(x) \nabla (u-u_h) \,dx \bigg)^{1/2}.
$$
Provide a similar visualization of the optimal convergence rates (in this norm) as in reference implementation from class for the $P_k$-FEM, $k = 1, 2, 3$.

In [ ]:
using ForwardDiff 

# generate a smooth solution consistent with the boundary condition 

u_ex = x ->  # choose a model "manufactured solution"
∇u_ex = x -> ForwardDiff.gradient(u_ex, x)
f_ex = x ->  - tr( ForwardDiff.jacobian(x -> A_fun(x) * ∇u_ex(x), x) )



---

The next two questions concern semi-empirical and theoretical convergence studies on domains with corners. No rigorous proofs need to be given, but please do justify your arguments. You can re-use the code from the class on corner-singularities almost verbatim, very few changes need to be made. 

In [ ]:
include("ferrite_tools.jl")
include("aitken.jl")

function compute_energy(k, N, generate_dom, ffun)
    grid, ∂Ω = generate_dom(N)
    cellvalues, dh, ch, K = setup_fem(k, grid, ∂Ω)
    K, f = assemble_global!(cellvalues, dh, K, ξ -> ffun(ξ));
    apply!(K, f, ch)
    u = K \ f;
    return 0.5 * dot(u, K * u) - dot(u, f)
end

## Q3: Regularity in Polygonal Domains - Part 1  [10]

We consider the boundary value problem 
$$\begin{aligned}
    -\Delta u &= 10, \quad x \in \Omega \\ 
        u &= 0, \quad x \in \partial\Omega
\end{aligned}$$
where $\Omega$ is a domain with a triangle cut out with re-entrant corner of angle $\pi/4$. 

What is the expected rate of convergence in energy-norm for P1-FEM and P2-FEM? Briefly justify your answer, then demonstrate this numerically.

In [ ]:
grid, ∂Ω = generate_wedge(20)

FerriteViz.wireframe(grid, markersize = 10, strokewidth = 2, 
                     figure = (resolution=(500,500),))

### Solution: theory

A the re-entrant corner with interior angle $\theta_0 = 7 \pi/4 = \pi/\alpha$ we have the leading-order singularity 
$$
    s_{\alpha} = r^\alpha \sin(\alpha \theta),
$$
with $\alpha = 4/7$. Following class notes we expect the rate of convergence in energy-norm to be only $O(h^{4/7})$ for both P1 and P2-fem. For convergence in energy we therefore get $O(h^{8/7})$.  This is fully supported by theory.

## Q4: Regularity in Polygonal Domains - Part 2  [10]

We consider the boundary value problem 
$$\begin{aligned}
    -\Delta u &= 5, \quad x \in \Omega \\ 
        u &= 0, \quad x \in \partial\Omega
\end{aligned}$$
where $\Omega$ is now again the unit square, namely a triangle with angles $\pi/4, \pi/4,  \pi/2$. (visualized in the first cell below)

What is the expected rate of convergence in energy-norm for P1-FEM, P2-FEM, P3-FEM? 

Even though the square should be the simplest of all cases, it is actually not trivial to adapt our arguments for re-entrant corners to this setting, even formally. Try to work it out, but if you can't see how to go about it, then do a purely empirical study and try to explain more intuitively what you observe. I will give close to full points for that.

(Because of the high symmetry, there are alternative techniques to understand the regularity of solutions in a square, e.g. Fourier. You could think about that too. But this takes a lot of time and I don't expect it.)

### Solution: theory

The domain is convex so $u \in H^2$, this means the rate for P1-FEM is $h$. 

But it is not clear whether $u \in H^3$ or even $H^4$ to get the optimal rates for P2 and P3? We look at the possible corner singularities: Here the interior angle at all corners is $\theta_0 = \pi/2 = \pi/\alpha$, i.e. $\alpha = 2$. This suggests the leading order corner singularity $s_2(x) = r^2 \sin(2 \theta)$. 

Because higher $r$-derivatives just vanish, we look at the $\theta$-derivatives instead. Recall that the derivatives in euclidean coordinates scale roughly as $\nabla_{xy} \sim (\partial_r, r^{-1} \partial_\theta)$. For the third derivative we get 
$$\begin{aligned}
    r^{-3} \partial_\theta^3 s_2 &= - 8 r^{-1} \cos(2\theta) 
\end{aligned}$$
This suggests that $|\nabla^3 s_2|^2 \approx r^{-2}$ near the singularity, which would mean that it is "almost but not quite integrable", i.e. $u \not\in H^3$, but "almost". 

We can speculate that the rate of convergence for P2 will be $h^t$ for all $t < k$, i.e. almost optimal. This is really hard to see numerically so we will test against $h^2$. For P3 to get a better rate we would need strictly better than $H^3$-regularity, which we clearlly cannot have, so the rate for P3 will be the same as for P2, i.e. almost $h^2$. (or, $h^4$ for the convergence of the energy)
